# Stock Price and Earnings Event Analysis
This notebook prepares daily stock price data and earnings announcement dates for Apple, Google, and Nvidia (2020–2025) and calculates daily returns in preparation for further analysis.

The dataset is stored as event_study_price_data.csv.

## 1. Load data
Two data sets are loaded: 
- daily stock price data (open, high, low, close) for Apple, Google, and Nvidia
- earnings announcement dates 

In [1]:
import pandas as pd

In [2]:
prices = pd.read_csv("stock_prices_and_earnings_dates/ohlc.csv")
earnings = pd.read_csv("stock_prices_and_earnings_dates/earnings_dates.csv")

In [3]:
prices.head()

,ticker,date,open,high,low,close,volume
0,AAPL,2020/01/02,74.06,75.15,73.80,75.09,33911864
1,AAPL,2020/01/03,74.29,75.15,74.13,74.36,36633878
2,AAPL,2020/01/06,73.45,74.99,73.19,74.95,29644644
3,AAPL,2020/01/07,74.96,75.22,74.37,74.60,27877655
4,AAPL,2020/01/08,74.29,76.11,74.29,75.80,33090946


In [4]:
earnings.head()

,ticker,earnings_date
0,AAPL,2020/01/28
1,AAPL,2020/04/30
2,AAPL,2020/07/30
3,AAPL,2020/10/29
4,AAPL,2021/01/27


## 2. Data type preparation
Date columns are converted to datetime  format to enable time-based calculations. 

In [5]:
# Store date columns as datetime objects.
prices['date'] = pd.to_datetime(prices['date'])

In [6]:
# Check data types
prices.dtypes

ticker            object
date      datetime64[ns]
open             float64
high             float64
low              float64
close            float64
volume             int64
dtype: object

In [7]:
earnings['earnings_date'] = pd.to_datetime(earnings['earnings_date'])

In [8]:
earnings.dtypes

ticker                   object
earnings_date    datetime64[ns]
dtype: object

## 3. Merge earnings dates with stock price data
In this section, the dataset is prepared for event-study analysis by merging earnings announcement dates with daily stock price data. We first ensure that price data are sorted chronologically within each firm. Earnings dates are then merged into the price dataset so that the stock price on each earnings announcement date can be identified.

In [9]:
# Sort data appropriately.
prices = prices.sort_values(['ticker', 'date'])
earnings = earnings.sort_values(['ticker', 'earnings_date'])

In [10]:
prices[['ticker','date']].head(5)

,ticker,date
0,AAPL,2020-01-02
1,AAPL,2020-01-03
2,AAPL,2020-01-06
3,AAPL,2020-01-07
4,AAPL,2020-01-08


In [11]:
earnings.head(10)

,ticker,earnings_date
0,AAPL,2020-01-28
1,AAPL,2020-04-30
2,AAPL,2020-07-30
3,AAPL,2020-10-29
4,AAPL,2021-01-27
5,AAPL,2021-04-28
6,AAPL,2021-07-27
7,AAPL,2021-10-28
8,AAPL,2022-01-27
9,AAPL,2022-04-28


In [12]:
# Merge earnings dates into prices.
prices = prices.merge(earnings, left_on=['ticker', 'date'], right_on=['ticker', 'earnings_date'], how='left')

In [13]:
# Check earnings dates have merged.
prices[['ticker','date','earnings_date']].dropna().head()

,ticker,date,earnings_date
17,AAPL,2020-01-28,2020-01-28
82,AAPL,2020-04-30,2020-04-30
145,AAPL,2020-07-30,2020-07-30
209,AAPL,2020-10-29,2020-10-29
269,AAPL,2021-01-27,2021-01-27


In [14]:
# After sorting by ticker and date, we assign each row a sequential trading-day number (td_num) within each ticker, 
# in order to identify days before and after an earnings event based on trading-day position rather than calendar dates.
prices = prices.sort_values(['ticker','date']).reset_index(drop=True)
prices['td_num'] = prices.groupby('ticker').cumcount()

In [15]:
prices[['ticker','date','td_num']].head(5)

,ticker,date,td_num
0,AAPL,2020-01-02,0
1,AAPL,2020-01-03,1
2,AAPL,2020-01-06,2
3,AAPL,2020-01-07,3
4,AAPL,2020-01-08,4


## 4. Construct Earnings Event Windows (Trading Days)
We create an event-day index for each earnings announcement. Day 0 represents the earnings announcement date. Trading days before and after the event are labelled in relation to this date (-5 to +5), enabling analysis of return behaviour around earnings events.

In [16]:
# Create an "events" table containing only earnings days and their trading-day position
events = prices.loc[
    prices['earnings_date'].notna(),
    ['ticker', 'earnings_date', 'td_num']
].rename(columns={'td_num': 'event_td_num'})

events.head(5)

,ticker,earnings_date,event_td_num
17,AAPL,2020-01-28,17
82,AAPL,2020-04-30,82
145,AAPL,2020-07-30,145
209,AAPL,2020-10-29,209
269,AAPL,2021-01-27,269


In [17]:
# Define the event window offsets (-5 to +5 trading days)
window = pd.DataFrame({'event_day': range(-5, 6)})
window

,event_day
0,-5
1,-4
2,-3
3,-2
4,-1
5,0
6,1
7,2
8,3
9,4


In [18]:
# Expand each earnings event into -5...+5 event-day rows
events_expanded = events.merge(window, how='cross')

# Compute the trading-day number for each event-day offset
events_expanded['td_num'] = events_expanded['event_td_num'] + events_expanded['event_day']

events_expanded.head(11)

,ticker,earnings_date,event_td_num,event_day,td_num
0,AAPL,2020-01-28,17,-5,12
1,AAPL,2020-01-28,17,-4,13
2,AAPL,2020-01-28,17,-3,14
3,AAPL,2020-01-28,17,-2,15
4,AAPL,2020-01-28,17,-1,16
5,AAPL,2020-01-28,17,0,17
6,AAPL,2020-01-28,17,1,18
7,AAPL,2020-01-28,17,2,19
8,AAPL,2020-01-28,17,3,20
9,AAPL,2020-01-28,17,4,21


In [33]:
# Convert event windows into real calendar dates
event_day_labels = events_expanded.merge(
    prices[['ticker', 'td_num', 'date']],  
    on=['ticker', 'td_num'],
    how='left'
)[['ticker', 'date', 'event_day']]

event_day_labels.head(11)

,ticker,date,event_day
0,AAPL,2020-01-21,-5
1,AAPL,2020-01-22,-4
2,AAPL,2020-01-23,-3
3,AAPL,2020-01-24,-2
4,AAPL,2020-01-27,-1
5,AAPL,2020-01-28,0
6,AAPL,2020-01-29,1
7,AAPL,2020-01-30,2
8,AAPL,2020-01-31,3
9,AAPL,2020-02-03,4


In [34]:
# Attach event-day labels to the full trading history
prices = prices.merge(
    event_day_labels,
    on=['ticker', 'date'],
    how='left'
)

prices.head(11)

,ticker,date,open,high,low,close,volume,earnings_date,td_num,event_day
0,AAPL,2020-01-02,74.06,75.15,73.80,75.09,33911864,NaT,0,NaN
1,AAPL,2020-01-03,74.29,75.15,74.13,74.36,36633878,NaT,1,NaN
2,AAPL,2020-01-06,73.45,74.99,73.19,74.95,29644644,NaT,2,NaN
3,AAPL,2020-01-07,74.96,75.22,74.37,74.60,27877655,NaT,3,NaN
4,AAPL,2020-01-08,74.29,76.11,74.29,75.80,33090946,NaT,4,NaN
5,AAPL,2020-01-09,76.81,77.61,76.55,77.41,42621542,NaT,5,NaN
6,AAPL,2020-01-10,77.65,78.17,77.06,77.58,35217272,NaT,6,NaN
7,AAPL,2020-01-13,77.91,79.27,77.79,79.24,30521722,NaT,7,NaN
8,AAPL,2020-01-14,79.18,79.39,78.04,78.17,40653457,NaT,8,NaN
9,AAPL,2020-01-15,77.96,78.88,77.39,77.83,30480882,NaT,9,NaN


## 5. Calculate daily returns

In [35]:
prices = prices.sort_values(['ticker', 'date'])

prices['daily_return'] = (
    prices.groupby('ticker')['close']
    .pct_change()
)

prices['daily_return_pct'] = prices['daily_return'] * 100

prices.head(11)

,ticker,date,open,high,low,close,volume,earnings_date,td_num,event_day,daily_return,daily_return_pct
0,AAPL,2020-01-02,74.06,75.15,73.80,75.09,33911864,NaT,0,NaN,NaN,NaN
1,AAPL,2020-01-03,74.29,75.15,74.13,74.36,36633878,NaT,1,NaN,-0.009722,-0.972167
2,AAPL,2020-01-06,73.45,74.99,73.19,74.95,29644644,NaT,2,NaN,0.007934,0.793437
3,AAPL,2020-01-07,74.96,75.22,74.37,74.60,27877655,NaT,3,NaN,-0.004670,-0.466978
4,AAPL,2020-01-08,74.29,76.11,74.29,75.80,33090946,NaT,4,NaN,0.016086,1.608579
5,AAPL,2020-01-09,76.81,77.61,76.55,77.41,42621542,NaT,5,NaN,0.021240,2.124011
6,AAPL,2020-01-10,77.65,78.17,77.06,77.58,35217272,NaT,6,NaN,0.002196,0.219610
7,AAPL,2020-01-13,77.91,79.27,77.79,79.24,30521722,NaT,7,NaN,0.021397,2.139727
8,AAPL,2020-01-14,79.18,79.39,78.04,78.17,40653457,NaT,8,NaN,-0.013503,-1.350328
9,AAPL,2020-01-15,77.96,78.88,77.39,77.83,30480882,NaT,9,NaN,-0.004349,-0.434949


## 6. Save as CSV file

In [36]:
# Save the DataFrame as a CSV file
prices.to_csv("event_study_price_data.csv", index=False)